In [18]:
import pyodbc
import pandas as pd
from pathlib import Path
import glob
import datetime
import re
import os
import numpy as np
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from datetime import date
import math
import time

In [19]:
# home and time
home = Path.home()
todaystr = date.today().strftime('%Y-%m-%d')

In [20]:
FD_path = Path( home, "desktop" , 'FD_all.xlsx')
FD = pd.read_excel( FD_path, sheet_name='Sheet1' )

In [21]:
start_time = time.time()
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()
for index, row in FD.iterrows():
    ODM = row['ODM']
    Item = row['Item']
    Commodity = row['Commodity']
    FV = row['FV']
    Platform = row['Platform']
    HP_PN = row['HP_PN']
    Supplier = row['Supplier']
    PN = row['HP PN']
    ReportDate = row['ReportDate']
    FDdate = row['FDdate']
    FDQty = row['FDQty']
    BuyerName = row['BuyerName']

    cursor.execute(f"INSERT INTO CSI.GPS.GPS_tbl_ops_fd_temp ( ODM,Item,Commodity,FV,Platform,Supplier,[HP PN],FDdate,FDQty,Reportdate,BuyerName )\
                    VALUES('{ODM}','{Item}','{Commodity}','{FV}','{Platform}','{Supplier}','{PN}','{FDdate}','{FDQty}','{ReportDate}','{BuyerName}')")
conn.commit()
conn.close()
print("%s seconds ---" % (time.time() - start_time))

104.62508249282837 seconds ---


In [22]:
import urllib
from sqlalchemy import create_engine

In [23]:
start_time = time.time()

server = 'g7w11206g.inc.hpicorp.net'
database = 'CSI'

# engine = create_engine('mssql+pyodbc://' + server + '/' + database + '?trusted_connection=yes&driver=SQL Server Native Client 11.0')
engine = create_engine('mssql+pyodbc://g7w11206g.inc.hpicorp.net/CSI?trusted_connection=yes&driver=SQL Server Native Client 11.0')


# urllib.parse.quote_plus(f'DRIVER={{SQL Server Native Client 11.0}};Server=g7w11206g.inc.hpicorp.net;Database=CSI')
# engine = create_engine(f'mssql+pyodbc:///?odbc_connect={connect_string}', fast_executemany=True)

# with engine.connect() as connection:
#     FD.to_sql(name ='CSI.GPS.GPS_tbl_ops_fd_temp', connection, index=False)
engine.connect()


In [24]:
FD.BuyerName.unique()

array(['ZhaoGinger(CW)', 'WangYu-Ting', 'ChenJasmine',
       'LeeCynthia(GPSTW)', 'TangWinnie', 'FerngApple', 'LinSharon',
       'Anny', 'LiSophia(GPSTW)', 'LaiShirley', 'Celine', 'Karen',
       'Ariel', 'LiangLavinia', 'Rocky', 'ChouSally(GPSTW)',
       'LeeLydia(GPSTaiwan)', 'HsuDustin', 'JhangEmily', 'PANJESSE',
       'Jennifer', 'ChenJosephine(GPS)', 'ChenVivian(GPS)', 'HsiehAngela'],
      dtype=object)

In [30]:
# FD.to_sql('[CSI].[GPS].[GPS_tbl_ops_fd_temp]', con = engine)

with engine.begin() as connection:
    FD.to_sql(name ='CSI.GPS.GPS_tbl_ops_fd_temp', schema = 'guest', con = connection, index=False, if_exists='append')


print("%s seconds ---" % (time.time() - start_time))

ProgrammingError: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][SQL Server Native Client 11.0][SQL Server]CREATE TABLE permission denied in database 'CSI'. (262) (SQLExecDirectW)")
[SQL: 
CREATE TABLE [public].[CSI.GPS.GPS_tbl_ops_fd_temp] (
	[ODM] VARCHAR(max) NULL, 
	[Item] VARCHAR(max) NULL, 
	[Commodity] VARCHAR(max) NULL, 
	[FV] VARCHAR(max) NULL, 
	[Platform] VARCHAR(max) NULL, 
	[HP_PN] VARCHAR(max) NULL, 
	[Supplier] VARCHAR(max) NULL, 
	[HP PN] VARCHAR(max) NULL, 
	[ReportDate] DATETIME NULL, 
	[FDdate] DATETIME NULL, 
	[FDQty] BIGINT NULL, 
	[BuyerName] VARCHAR(max) NULL
)

]
(Background on this error at: https://sqlalche.me/e/14/f405)